In [131]:
import pandas as pd
import re
import numpy as np
import numpy.matlib
import nest_asyncio
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<h1>Data Initialization</h1>
<h3>Append all 11 scrape results</h3>

In [40]:
#Read all 11 scrape
df1 = pd.read_csv('tokopedia_data.csv')
df2 = pd.read_csv('tokopedia_data_2.csv')
df3 = pd.read_csv('tokopedia_data_3.csv')
df4 = pd.read_csv('tokopedia_data_4.csv')
df5 = pd.read_csv('tokopedia_data_5.csv')
df6 = pd.read_csv('tokopedia_data_6.csv')
df7 = pd.read_csv('tokopedia_data_7.csv')
df8 = pd.read_csv('tokopedia_data_8.csv')
df9 = pd.read_csv('tokopedia_data_9.csv')
df10 = pd.read_csv('tokopedia_data_10.csv')
df11 = pd.read_csv('tokopedia_data_11.csv')

In [41]:
#Concatanate vertically
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], ignore_index=True, sort=False)

In [42]:
df_tokopedia = df.copy()
df_tokopedia

,product_name,rating,category,from_where,satisfaction_rate,img_link,description,amount_sold,original_price,product_price,five_star,four_star,three_star,two_star,one_star
0,Al-Quran Ash-Shahib ORI A5 - Merah,5,Buku,Jakarta Timur,99%,https://images.tokopedia.net/img/cache/500-squ...,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,Terjual 2 rb+,Rp75.000,Rp75.000,657,25,4,0,0
1,MASKER ANAK KF94 KOREA KIDS FACE MASK KF 94 AN...,4.9,Fashion Anak & Bayi,Jakarta Barat,99%,https://images.tokopedia.net/img/cache/500-squ...,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,Terjual 10 rb+,Rp6.500,Rp5.000,1621,104,9,1,3
2,Otogard GOLD LINE Wiper Fluid Concentrate,4.9,Otomotif,Jakarta Selatan,97%,https://images.tokopedia.net/img/cache/500-squ...,- Cairan konsentrat yang berfungsi untuk membe...,Terjual 4 rb+,Rp72.000,Rp36.000,2339,145,34,4,24
3,Detick 1ml 2ml Obat Kutu Tetes Anjing Kucing 1...,4.9,Perawatan Hewan,Kab. Tangerang,98%,https://images.tokopedia.net/img/cache/500-squ...,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,Terjual 10 rb+,Rp24.000,Rp16.950,1161,76,15,1,6
4,Bebek Kloset Ekstra Kuat 450mL,4.8,Pertukangan,Gudang Terdekat,96%,https://images.tokopedia.net/img/cache/500-squ...,Bebek Kloset merupakan pembersih kloset yang m...,Terjual 5 rb+,Rp21.500,Rp18.920,1102,57,21,6,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,Safety Traffic Cone / Kerucut Lalu Lintas Tona...,5,Pertukangan,Kab. Tangerang,99%,https://images.tokopedia.net/img/cache/500-squ...,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,Terjual 10 rb+,Rp88.000,Rp88.000,1453,51,4,1,1
5045,PAKET COD (PENTING),5,Buku,Kab. Bandung Barat,100,https://images.tokopedia.net/img/cache/500-squ...,Untuk Info silahka Banyak produk yang bisa di...,Terjual 8 rb+,Rp1.000,Rp1.000,29,0,0,0,0
5046,Game Boy Retro FC 400 in 1 Gameboy Mini Gamepa...,4.8,Gaming,Kota Tangerang,95%,https://images.tokopedia.net/img/cache/500-squ...,Link Gameboy + Stick : https://www.tokopedia.c...,Terjual 10 rb+,Rp72.500,Rp72.500,1654,104,46,10,21
5047,SENDOK TEH ISI 12 PCS - STAINLESS STEEL,4.9,Dapur,Jakarta Barat,96%,https://images.tokopedia.net/img/cache/500-squ...,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,Terjual 1 rb+,Rp8.000,Rp8.000,229,18,6,1,1


In [43]:
df_tokopedia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5049 entries, 0 to 5048
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_name       5049 non-null   object
 1   rating             5049 non-null   object
 2   category           5049 non-null   object
 3   from_where         5049 non-null   object
 4   satisfaction_rate  5049 non-null   object
 5   img_link           5049 non-null   object
 6   description        5029 non-null   object
 7   amount_sold        5049 non-null   object
 8   original_price     5049 non-null   object
 9   product_price      5049 non-null   object
 10  five_star          5049 non-null   object
 11  four_star          5049 non-null   int64 
 12  three_star         5049 non-null   int64 
 13  two_star           5049 non-null   int64 
 14  one_star           5049 non-null   int64 
dtypes: int64(4), object(11)
memory usage: 591.8+ KB


In [53]:
df_tokopedia.drop_duplicates(subset=['product_name'], inplace=True)
df_tokopedia.duplicated(subset=['product_name']).sum()

0

<h2>Satisfaction Rate Fix</h2>

In [190]:
df_tokopedia.loc[df_tokopedia['satisfaction_rate'] == '100', 'satisfaction_rate'] = '100%'
# display(df_tokopedia.iloc[[5045]]['satisfaction_rate'][5045])

'100%'

<h2>Description Pre-Processing</h2>

<b>Cleaning, Tokenizing, Removing Stop Words, Stemming the description</b>

In [45]:
df_tokopedia['cleaned_description'] = [str(x).lower() for x in df_tokopedia['description']]

In [46]:
df_tokopedia['cleaned_description'] = [re.sub(r"www.\S+", " ", str(x)) for x in df_tokopedia['cleaned_description']]
df_tokopedia['cleaned_description'] = [re.sub(r"http\S+", " ", str(x)) for x in df_tokopedia['cleaned_description']]
df_tokopedia['cleaned_description'] = [re.sub("[^a-zA-Z0-9]", " ", str(x)) for x in df_tokopedia['cleaned_description']]
df_tokopedia['cleaned_description'] = [re.sub("\s\s+", " ", str(x)) for x in df_tokopedia['cleaned_description']]

df_tokopedia[['description','cleaned_description']]

,description,cleaned_description
0,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,perhatian semua buku di toko harga buku origin...
1,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,masker earloop kf94 anaksudah izin kemenkes ri...
2,- Cairan konsentrat yang berfungsi untuk membe...,cairan konsentrat yang berfungsi untuk member...
3,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,detick 1ml 2ml obat kutu tetes anjing kucing h...
4,Bebek Kloset merupakan pembersih kloset yang m...,bebek kloset merupakan pembersih kloset yang m...
...,...,...
5044,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,ready stock tonata rubber cone 1 8 kgsspesifik...
5045,Untuk Info silahka Banyak produk yang bisa di...,untuk info silahka banyak produk yang bisa di ...
5046,Link Gameboy + Stick : https://www.tokopedia.c...,link gameboy stick stick nya saja 5 warna hite...
5047,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,sendok teh isi 12 pcs panjang 14cmstainless steel


In [47]:
df_tokopedia['cleaned_description'] = df_tokopedia.apply(lambda row: word_tokenize(row['cleaned_description']), axis=1)
df_tokopedia[['description', 'cleaned_description']]

,description,cleaned_description
0,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,"[perhatian, semua, buku, di, toko, harga, buku..."
1,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,"[masker, earloop, kf94, anaksudah, izin, kemen..."
2,- Cairan konsentrat yang berfungsi untuk membe...,"[cairan, konsentrat, yang, berfungsi, untuk, m..."
3,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,"[detick, 1ml, 2ml, obat, kutu, tetes, anjing, ..."
4,Bebek Kloset merupakan pembersih kloset yang m...,"[bebek, kloset, merupakan, pembersih, kloset, ..."
...,...,...
5044,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,"[ready, stock, tonata, rubber, cone, 1, 8, kgs..."
5045,Untuk Info silahka Banyak produk yang bisa di...,"[untuk, info, silahka, banyak, produk, yang, b..."
5046,Link Gameboy + Stick : https://www.tokopedia.c...,"[link, gameboy, stick, stick, nya, saja, 5, wa..."
5047,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,"[sendok, teh, isi, 12, pcs, panjang, 14cmstain..."


In [48]:
nltk.download('stopwords')
stop_words = set(stopwords.words("indonesian"))
df_tokopedia['cleaned_description'] = df_tokopedia['cleaned_description'].apply(lambda row: [word for word in row if word not in stop_words])
df_tokopedia[['description', 'cleaned_description']]

[nltk_data] Downloading package stopwords to C:\Users\Kenny
[nltk_data]     Jinhiro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,description,cleaned_description
0,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,"[perhatian, buku, toko, harga, buku, original,..."
1,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,"[masker, earloop, kf94, anaksudah, izin, kemen..."
2,- Cairan konsentrat yang berfungsi untuk membe...,"[cairan, konsentrat, berfungsi, membersihan, k..."
3,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,"[detick, 1ml, 2ml, obat, kutu, tetes, anjing, ..."
4,Bebek Kloset merupakan pembersih kloset yang m...,"[bebek, kloset, pembersih, kloset, mengandung,..."
...,...,...
5044,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,"[ready, stock, tonata, rubber, cone, 1, 8, kgs..."
5045,Untuk Info silahka Banyak produk yang bisa di...,"[info, silahka, produk, beli]"
5046,Link Gameboy + Stick : https://www.tokopedia.c...,"[link, gameboy, stick, stick, nya, 5, warna, h..."
5047,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,"[sendok, teh, isi, 12, pcs, 14cmstainless, steel]"


In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df_tokopedia['cleaned_description'] = df_tokopedia['cleaned_description'].apply(lambda row: stemmer.stem(' '.join(word for word in row)).split())
df_tokopedia[['description', 'cleaned_description']]

<h1>Title Pre-Processing</h1>

In [ ]:
df_tokopedia['cleaned_product_name'] = [str(x).lower() for x in df_tokopedia['product_name']]

In [ ]:
df_tokopedia['cleaned_product_name'] = [re.sub(r"www.\S+", " ", str(x)) for x in df_tokopedia['cleaned_product_name']]
df_tokopedia['cleaned_product_name'] = [re.sub(r"http\S+", " ", str(x)) for x in df_tokopedia['cleaned_product_name']]
df_tokopedia['cleaned_product_name'] = [re.sub("[^a-zA-Z0-9]", " ", str(x)) for x in df_tokopedia['cleaned_product_name']]
df_tokopedia['cleaned_product_name'] = [re.sub("\s\s+", " ", str(x)) for x in df_tokopedia['cleaned_product_name']]

df_tokopedia[['product_name','cleaned_product_name']]

In [ ]:
df_tokopedia['cleaned_product_name'] = df_tokopedia.apply(lambda row: word_tokenize(row['cleaned_product_name']), axis=1)
df_tokopedia[['product_name', 'cleaned_product_name']]

In [ ]:
df_tokopedia['cleaned_product_name'] = df_tokopedia['cleaned_product_name'].apply(lambda row: [word for word in row if word not in stop_words])
df_tokopedia[['product_name', 'cleaned_product_name']]

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df_tokopedia['cleaned_product_name'] = df_tokopedia['cleaned_product_name'].apply(lambda row: stemmer.stem(' '.join(word for word in row)).split())
df_tokopedia[['product_name', 'cleaned_product_name']]

<h2>Category Pre-processing</h2>

In [147]:
df_tokopedia['category']

0                      Buku
1       Fashion Anak & Bayi
2                  Otomotif
3           Perawatan Hewan
4               Pertukangan
               ...         
5044            Pertukangan
5045                   Buku
5046                 Gaming
5047                  Dapur
5048                 Gaming
Name: category, Length: 5029, dtype: object

In [174]:
df_tokopedia['tagged_category'] = 'kategori-'+df_tokopedia['category'].str.lower()

In [175]:
df_tokopedia['tagged_category'] = df_tokopedia['tagged_category'].str.replace(' ', '')

In [176]:
df_tokopedia['tagged_category'] = df_tokopedia['tagged_category'].str.replace('&', '-')
df_tokopedia['tagged_category']

0                   kategori-buku
1       kategori-fashionanak-bayi
2               kategori-otomotif
3         kategori-perawatanhewan
4            kategori-pertukangan
                  ...            
5044         kategori-pertukangan
5045                kategori-buku
5046              kategori-gaming
5047               kategori-dapur
5048              kategori-gaming
Name: tagged_category, Length: 5049, dtype: object

In [177]:
df_tokopedia

,product_name,rating,category,from_where,satisfaction_rate,img_link,description,amount_sold,original_price,product_price,five_star,four_star,three_star,two_star,one_star,cleaned_description,cleaned_product_name,tagged_category
0,Al-Quran Ash-Shahib ORI A5 - Merah,5,Buku,Jakarta Timur,99%,https://images.tokopedia.net/img/cache/500-squ...,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,Terjual 2 rb+,Rp75.000,Rp75.000,657,25,4,0,0,perhati buku toko harga buku original asli lan...,al quran ash shahib ori a5 merah,kategori-buku
1,MASKER ANAK KF94 KOREA KIDS FACE MASK KF 94 AN...,4.9,Fashion Anak & Bayi,Jakarta Barat,99%,https://images.tokopedia.net/img/cache/500-squ...,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,Terjual 10 rb+,Rp6.500,Rp5.000,1621,104,9,1,3,masker earloop kf94 anaksudah izin kemenkes ri...,masker anak kf94 korea kids face mask kf 94 an...,kategori-fashionanak-bayi
2,Otogard GOLD LINE Wiper Fluid Concentrate,4.9,Otomotif,Jakarta Selatan,97%,https://images.tokopedia.net/img/cache/500-squ...,- Cairan konsentrat yang berfungsi untuk membe...,Terjual 4 rb+,Rp72.000,Rp36.000,2339,145,34,4,24,cair konsentrat fungsi bersih kaca noda kotor ...,otogard gold line wiper fluid concentrate,kategori-otomotif
3,Detick 1ml 2ml Obat Kutu Tetes Anjing Kucing 1...,4.9,Perawatan Hewan,Kab. Tangerang,98%,https://images.tokopedia.net/img/cache/500-squ...,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,Terjual 10 rb+,Rp24.000,Rp16.950,1161,76,15,1,6,detick 1ml 2ml obat kutu tetes anjing kucing h...,detick 1ml 2ml obat kutu tetes anjing kucing 1...,kategori-perawatanhewan
4,Bebek Kloset Ekstra Kuat 450mL,4.8,Pertukangan,Gudang Terdekat,96%,https://images.tokopedia.net/img/cache/500-squ...,Bebek Kloset merupakan pembersih kloset yang m...,Terjual 5 rb+,Rp21.500,Rp18.920,1102,57,21,6,17,bebek kloset bersih kloset kandung germinex bu...,bebek kloset ekstra kuat 450ml,kategori-pertukangan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,Safety Traffic Cone / Kerucut Lalu Lintas Tona...,5,Pertukangan,Kab. Tangerang,99%,https://images.tokopedia.net/img/cache/500-squ...,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,Terjual 10 rb+,Rp88.000,Rp88.000,1453,51,4,1,1,ready stock tonata rubber cone 1 8 kgsspesifik...,safety traffic cone kerucut lintas tonata 1 8 ...,kategori-pertukangan
5045,PAKET COD (PENTING),5,Buku,Kab. Bandung Barat,100,https://images.tokopedia.net/img/cache/500-squ...,Untuk Info silahka Banyak produk yang bisa di...,Terjual 8 rb+,Rp1.000,Rp1.000,29,0,0,0,0,info silahka produk beli,paket cod,kategori-buku
5046,Game Boy Retro FC 400 in 1 Gameboy Mini Gamepa...,4.8,Gaming,Kota Tangerang,95%,https://images.tokopedia.net/img/cache/500-squ...,Link Gameboy + Stick : https://www.tokopedia.c...,Terjual 10 rb+,Rp72.500,Rp72.500,1654,104,46,10,21,link gameboy stick stick nya 5 warna hitem mer...,game boy retro fc 400 in 1 gameboy mini gamepa...,kategori-gaming
5047,SENDOK TEH ISI 12 PCS - STAINLESS STEEL,4.9,Dapur,Jakarta Barat,96%,https://images.tokopedia.net/img/cache/500-squ...,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,Terjual 1 rb+,Rp8.000,Rp8.000,229,18,6,1,1,sendok teh isi 12 pcs 14cmstainless steel,sendok teh isi 12 pcs stainless steel,kategori-dapur


<h2>From_where Pre-Processing</h2>

In [198]:
df_tokopedia['tagged_location'] = 'lokasi-'+df_tokopedia['from_where'].str.lower()

In [199]:
df_tokopedia['tagged_location'] = df_tokopedia['tagged_location'].str.replace(' ', '')

In [200]:
df_tokopedia['tagged_location']

0           lokasi-jakartatimur
1           lokasi-jakartabarat
2         lokasi-jakartaselatan
3          lokasi-kab.tangerang
4         lokasi-gudangterdekat
                 ...           
5044       lokasi-kab.tangerang
5045    lokasi-kab.bandungbarat
5046       lokasi-kotatangerang
5047        lokasi-jakartabarat
5048          lokasi-kotabekasi
Name: tagged_location, Length: 5049, dtype: object

<h2>Finalizing list to string</h2>

In [ ]:
df_tokopedia['cleaned_product_name'] = df_tokopedia['cleaned_product_name'].apply(eval).apply(' '.join)
df_tokopedia['cleaned_description'] = df_tokopedia['cleaned_description'].apply(eval).apply(' '.join)

In [203]:
df_tokopedia['product_tag'] = df_tokopedia['cleaned_description'] + df_tokopedia['cleaned_product_name'] + df_tokopedia['tagged_category'] + df_tokopedia['tagged_location']
df_tokopedia

,product_name,rating,category,from_where,satisfaction_rate,img_link,description,amount_sold,original_price,product_price,five_star,four_star,three_star,two_star,one_star,cleaned_description,cleaned_product_name,tagged_category,tagged_location,product_tag
0,Al-Quran Ash-Shahib ORI A5 - Merah,5,Buku,Jakarta Timur,99%,https://images.tokopedia.net/img/cache/500-squ...,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,Terjual 2 rb+,Rp75.000,Rp75.000,657,25,4,0,0,perhati buku toko harga buku original asli lan...,al quran ash shahib ori a5 merah,kategori-buku,lokasi-jakartatimur,perhati buku toko harga buku original asli lan...
1,MASKER ANAK KF94 KOREA KIDS FACE MASK KF 94 AN...,4.9,Fashion Anak & Bayi,Jakarta Barat,99%,https://images.tokopedia.net/img/cache/500-squ...,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,Terjual 10 rb+,Rp6.500,Rp5.000,1621,104,9,1,3,masker earloop kf94 anaksudah izin kemenkes ri...,masker anak kf94 korea kids face mask kf 94 an...,kategori-fashionanak-bayi,lokasi-jakartabarat,masker earloop kf94 anaksudah izin kemenkes ri...
2,Otogard GOLD LINE Wiper Fluid Concentrate,4.9,Otomotif,Jakarta Selatan,97%,https://images.tokopedia.net/img/cache/500-squ...,- Cairan konsentrat yang berfungsi untuk membe...,Terjual 4 rb+,Rp72.000,Rp36.000,2339,145,34,4,24,cair konsentrat fungsi bersih kaca noda kotor ...,otogard gold line wiper fluid concentrate,kategori-otomotif,lokasi-jakartaselatan,cair konsentrat fungsi bersih kaca noda kotor ...
3,Detick 1ml 2ml Obat Kutu Tetes Anjing Kucing 1...,4.9,Perawatan Hewan,Kab. Tangerang,98%,https://images.tokopedia.net/img/cache/500-squ...,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,Terjual 10 rb+,Rp24.000,Rp16.950,1161,76,15,1,6,detick 1ml 2ml obat kutu tetes anjing kucing h...,detick 1ml 2ml obat kutu tetes anjing kucing 1...,kategori-perawatanhewan,lokasi-kab.tangerang,detick 1ml 2ml obat kutu tetes anjing kucing h...
4,Bebek Kloset Ekstra Kuat 450mL,4.8,Pertukangan,Gudang Terdekat,96%,https://images.tokopedia.net/img/cache/500-squ...,Bebek Kloset merupakan pembersih kloset yang m...,Terjual 5 rb+,Rp21.500,Rp18.920,1102,57,21,6,17,bebek kloset bersih kloset kandung germinex bu...,bebek kloset ekstra kuat 450ml,kategori-pertukangan,lokasi-gudangterdekat,bebek kloset bersih kloset kandung germinex bu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,Safety Traffic Cone / Kerucut Lalu Lintas Tona...,5,Pertukangan,Kab. Tangerang,99%,https://images.tokopedia.net/img/cache/500-squ...,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,Terjual 10 rb+,Rp88.000,Rp88.000,1453,51,4,1,1,ready stock tonata rubber cone 1 8 kgsspesifik...,safety traffic cone kerucut lintas tonata 1 8 ...,kategori-pertukangan,lokasi-kab.tangerang,ready stock tonata rubber cone 1 8 kgsspesifik...
5045,PAKET COD (PENTING),5,Buku,Kab. Bandung Barat,100%,https://images.tokopedia.net/img/cache/500-squ...,Untuk Info silahka Banyak produk yang bisa di...,Terjual 8 rb+,Rp1.000,Rp1.000,29,0,0,0,0,info silahka produk beli,paket cod,kategori-buku,lokasi-kab.bandungbarat,info silahka produk belipaket codkategori-buku...
5046,Game Boy Retro FC 400 in 1 Gameboy Mini Gamepa...,4.8,Gaming,Kota Tangerang,95%,https://images.tokopedia.net/img/cache/500-squ...,Link Gameboy + Stick : https://www.tokopedia.c...,Terjual 10 rb+,Rp72.500,Rp72.500,1654,104,46,10,21,link gameboy stick stick nya 5 warna hitem mer...,game boy retro fc 400 in 1 gameboy mini gamepa...,kategori-gaming,lokasi-kotatangerang,link gameboy stick stick nya 5 warna hitem mer...
5047,SENDOK TEH ISI 12 PCS - STAINLESS STEEL,4.9,Dapur,Jakarta Barat,96%,https://images.tokopedia.net/img/cache/500-squ...,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,Terjual 1 rb+,Rp8.000,Rp8.000,229,18,6,1,1,sendok teh isi 12 pcs 14cmstainless steel,sendok teh isi 12 pcs stainless steel,kategori-dapur,lokasi-jakartabarat,sendok teh isi 12 pcs 14cmstainless steelsendo...


In [205]:
df_tokopedia.to_csv('df_tokopedia_clean.csv', index=False)

<h1>Vectorize</h1>

In [219]:
df_tokopedia = pd.read_csv('df_tokopedia_clean.csv')
df_tokopedia

,product_name,rating,category,from_where,satisfaction_rate,img_link,description,amount_sold,original_price,product_price,five_star,four_star,three_star,two_star,one_star,cleaned_description,cleaned_product_name,tagged_category,tagged_location,product_tag
0,Al-Quran Ash-Shahib ORI A5 - Merah,5,Buku,Jakarta Timur,99%,https://images.tokopedia.net/img/cache/500-squ...,PERHATIAN :- Semua buku di TOKO HARGA BUKU ORI...,Terjual 2 rb+,Rp75.000,Rp75.000,657,25,4,0,0,perhati buku toko harga buku original asli lan...,al quran ash shahib ori a5 merah,kategori-buku,lokasi-jakartatimur,perhati buku toko harga buku original asli lan...
1,MASKER ANAK KF94 KOREA KIDS FACE MASK KF 94 AN...,4.9,Fashion Anak & Bayi,Jakarta Barat,99%,https://images.tokopedia.net/img/cache/500-squ...,MASKER EARLOOP KF94 ANAKSUDAH IZIN KEMENKES RI...,Terjual 10 rb+,Rp6.500,Rp5.000,1621,104,9,1,3,masker earloop kf94 anaksudah izin kemenkes ri...,masker anak kf94 korea kids face mask kf 94 an...,kategori-fashionanak-bayi,lokasi-jakartabarat,masker earloop kf94 anaksudah izin kemenkes ri...
2,Otogard GOLD LINE Wiper Fluid Concentrate,4.9,Otomotif,Jakarta Selatan,97%,https://images.tokopedia.net/img/cache/500-squ...,- Cairan konsentrat yang berfungsi untuk membe...,Terjual 4 rb+,Rp72.000,Rp36.000,2339,145,34,4,24,cair konsentrat fungsi bersih kaca noda kotor ...,otogard gold line wiper fluid concentrate,kategori-otomotif,lokasi-jakartaselatan,cair konsentrat fungsi bersih kaca noda kotor ...
3,Detick 1ml 2ml Obat Kutu Tetes Anjing Kucing 1...,4.9,Perawatan Hewan,Kab. Tangerang,98%,https://images.tokopedia.net/img/cache/500-squ...,DETICK 1ml 2ml Obat Kutu Tetes Anjing Kucing H...,Terjual 10 rb+,Rp24.000,Rp16.950,1161,76,15,1,6,detick 1ml 2ml obat kutu tetes anjing kucing h...,detick 1ml 2ml obat kutu tetes anjing kucing 1...,kategori-perawatanhewan,lokasi-kab.tangerang,detick 1ml 2ml obat kutu tetes anjing kucing h...
4,Bebek Kloset Ekstra Kuat 450mL,4.8,Pertukangan,Gudang Terdekat,96%,https://images.tokopedia.net/img/cache/500-squ...,Bebek Kloset merupakan pembersih kloset yang m...,Terjual 5 rb+,Rp21.500,Rp18.920,1102,57,21,6,17,bebek kloset bersih kloset kandung germinex bu...,bebek kloset ekstra kuat 450ml,kategori-pertukangan,lokasi-gudangterdekat,bebek kloset bersih kloset kandung germinex bu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5024,Safety Traffic Cone / Kerucut Lalu Lintas Tona...,5,Pertukangan,Kab. Tangerang,99%,https://images.tokopedia.net/img/cache/500-squ...,Ready Stock - Tonata Rubber Cone 1.8 KgsSpesif...,Terjual 10 rb+,Rp88.000,Rp88.000,1453,51,4,1,1,ready stock tonata rubber cone 1 8 kgsspesifik...,safety traffic cone kerucut lintas tonata 1 8 ...,kategori-pertukangan,lokasi-kab.tangerang,ready stock tonata rubber cone 1 8 kgsspesifik...
5025,PAKET COD (PENTING),5,Buku,Kab. Bandung Barat,100%,https://images.tokopedia.net/img/cache/500-squ...,Untuk Info silahka Banyak produk yang bisa di...,Terjual 8 rb+,Rp1.000,Rp1.000,29,0,0,0,0,info silahka produk beli,paket cod,kategori-buku,lokasi-kab.bandungbarat,info silahka produk belipaket codkategori-buku...
5026,Game Boy Retro FC 400 in 1 Gameboy Mini Gamepa...,4.8,Gaming,Kota Tangerang,95%,https://images.tokopedia.net/img/cache/500-squ...,Link Gameboy + Stick : https://www.tokopedia.c...,Terjual 10 rb+,Rp72.500,Rp72.500,1654,104,46,10,21,link gameboy stick stick nya 5 warna hitem mer...,game boy retro fc 400 in 1 gameboy mini gamepa...,kategori-gaming,lokasi-kotatangerang,link gameboy stick stick nya 5 warna hitem mer...
5027,SENDOK TEH ISI 12 PCS - STAINLESS STEEL,4.9,Dapur,Jakarta Barat,96%,https://images.tokopedia.net/img/cache/500-squ...,SENDOK TEH ISI 12 PCS Panjang 14cmSTAINLESS STEEL,Terjual 1 rb+,Rp8.000,Rp8.000,229,18,6,1,1,sendok teh isi 12 pcs 14cmstainless steel,sendok teh isi 12 pcs stainless steel,kategori-dapur,lokasi-jakartabarat,sendok teh isi 12 pcs 14cmstainless steelsendo...


In [204]:
df_tokopedia.dropna(inplace=True)
df_tokopedia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5029 entries, 0 to 5048
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   product_name          5029 non-null   object
 1   rating                5029 non-null   object
 2   category              5029 non-null   object
 3   from_where            5029 non-null   object
 4   satisfaction_rate     5029 non-null   object
 5   img_link              5029 non-null   object
 6   description           5029 non-null   object
 7   amount_sold           5029 non-null   object
 8   original_price        5029 non-null   object
 9   product_price         5029 non-null   object
 10  five_star             5029 non-null   object
 11  four_star             5029 non-null   int64 
 12  three_star            5029 non-null   int64 
 13  two_star              5029 non-null   int64 
 14  one_star              5029 non-null   int64 
 15  cleaned_description   5029 non-null   

In [215]:
def recommend(product_name):
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,6), min_df=0)
    product_index = df_tokopedia[df_tokopedia['product_name'] == product_name].index[0]

    tfidf_product_tag_vector = tfidf_vectorizer.fit_transform(df_tokopedia['product_tag'])
    product_tag_query = df_tokopedia.iloc[[product_index]]['product_tag'][product_index]
    product_query_vector = tfidf_vectorizer.transform([product_tag_query])
    product_similarity = cosine_similarity(product_query_vector, tfidf_product_tag_vector).flatten()

    # product_indices = np.argpartition(product_name_description_similarity, -1)[-1:]
    # product_indices = np.argpartition(product_name_similarity, -5)[-5:]
    product_indices = np.argpartition(product_similarity, -10)[-10:]
    product_results = df_tokopedia.iloc[product_indices]
    # product_name_
    return product_results

In [218]:
recommend('Pick Gitar atau Pemetik Gitar murah - 0.71mm')

,product_name,rating,category,from_where,satisfaction_rate,img_link,description,amount_sold,original_price,product_price,five_star,four_star,three_star,two_star,one_star,cleaned_description,cleaned_product_name,tagged_category,tagged_location,product_tag
907,Guitar Capo Kapo Gitar Bass Akustik Elektrik U...,4.8,Film & Musik,Kota Depok,96%,https://images.tokopedia.net/img/cache/500-squ...,"Bisa untuk gitar akustik, listrik, maupun gita...",Terjual 3 rb+,Rp12.500,Rp12.500,1399,135,39,10,9,gitar akustik listrik gitar klasik warna hitam...,guitar capo kapo gitar bass akustik elektrik u...,kategori-film-musik,lokasi-kotadepok,gitar akustik listrik gitar klasik warna hitam...
229,Stand Gitar - Tempat Gitar - Dudukan Gitar - T...,4.9,Film & Musik,Kota Tangerang,98%,https://images.tokopedia.net/img/cache/500-squ...,Height Adjustable: 64 - 75 cmMaterial: Metal +...,Terjual 500+,Rp62.500,Rp62.500,303,14,1,1,2,height adjustable 64 75 cmmaterial metal rubbe...,stand gitar gitar duduk gitar taffstudio hk00433,kategori-film-musik,lokasi-kotatangerang,height adjustable 64 75 cmmaterial metal rubbe...
2363,Pick Alice Original Pik Gitar Ukuran 0.46 0.71...,4.9,Film & Musik,Kota Bandung,98%,https://images.tokopedia.net/img/cache/500-squ...,Pick Alice ini adalah pick yang sudah dikenal ...,Terjual 3 rb+,Rp1.200,Rp1.200,149,8,1,0,1,pick alice pick kenal musisi produk original s...,pick alice original pik gitar ukur 0 46 0 71 0...,kategori-film-musik,lokasi-kotabandung,pick alice pick kenal musisi produk original s...
3500,Senar Gitar Elektrik String Ernie Ball Slinky ...,4.8,Film & Musik,Kota Bandung,95%,https://images.tokopedia.net/img/cache/500-squ...,READY STOCK kakk.. 😇 LANGSUNG ORDER YA KA....P...,Terjual 1 rb+,Rp80.000,Rp54.000,566,29,13,1,11,ready stock kakk langsung order ya ka pesan ke...,senar gitar elektrik string ernie ball slinky ...,kategori-film-musik,lokasi-kotabandung,ready stock kakk langsung order ya ka pesan ke...
4309,Alice Pick Gitar Akustik 50 pcs - A011A,5,Film & Musik,Kota Bandung,99%,https://images.tokopedia.net/img/cache/500-squ...,Pick Gitar Alice 50pcs + Box Warna: multi colo...,Terjual 250+,Rp29.999,Rp28.500,147,5,1,0,0,pick gitar alice 50pcs box warna multi color a...,alice pick gitar akustik 50 pcs a011a,kategori-film-musik,lokasi-kotabandung,pick gitar alice 50pcs box warna multi color a...
4489,Senar Gitar Elektrik String Elixir Polyweb Nan...,4.9,Film & Musik,Kota Bandung,97%,https://images.tokopedia.net/img/cache/500-squ...,READY STOCK kakk.. 😇 LANGSUNG ORDER YA KA....P...,Terjual 750+,Rp80.000,Rp54.000,375,25,7,2,2,ready stock kakk langsung order ya ka pesan ke...,senar gitar elektrik string elixir polyweb nan...,kategori-film-musik,lokasi-kotabandung,ready stock kakk langsung order ya ka pesan ke...
2332,Pick Gitar Akustik 50PCS - Multi-Color,4.9,Film & Musik,Jakarta Pusat,99%,https://images.tokopedia.net/img/cache/500-squ...,Spec of Pick Gitar Akustik 50PCS :Material \t:...,Terjual 5 rb+,Rp27.500,Rp27.500,1958,95,14,2,1,spec of pick gitar akustik 50pcs material plas...,pick gitar akustik 50pcs multi color,kategori-film-musik,lokasi-jakartapusat,spec of pick gitar akustik 50pcs material plas...
3861,Senar Gitar Akustik String Elixir Nanoweb 0.10...,4.9,Film & Musik,Kota Bandung,98%,https://images.tokopedia.net/img/cache/500-squ...,Note : Hati hati produk tiruan dengan harga mu...,Terjual 1 rb+,Rp75.000,Rp45.000,524,29,5,3,2,note hati hati produk tiru harga murah selisih...,senar gitar akustik string elixir nanoweb 0 10...,kategori-film-musik,lokasi-kotabandung,note hati hati produk tiru harga murah selisih...
3369,Alice Pick Gitar Akustik 50 PCS Multi Color,4.9,Film & Musik,Jakarta Barat,98%,https://images.tokopedia.net/img/cache/500-squ...,Pick Gitar atau Guitar Pick adalah sejenis ala...,Terjual 1 rb+,Rp55.100,Rp55.100,499,18,5,1,0,pick gitar guitar pick jenis alat bantu petik ...,alice pick gitar akustik 50 pcs multi color,kategori-film-musik,lokasi-jakartabarat,pick gitar guitar pick jenis alat bantu peti